<a href="https://colab.research.google.com/github/ZOUCH-AMIN/ABC-classification/blob/main/kmean_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
# lire les données dans une dataframe
df = pd.read_csv("https://raw.githubusercontent.com/ZOUCH-AMIN/ABC-classification/main/sales.csv")

In [ ]:
df["revenue"] = df["units_sold"] * df["price"]

In [ ]:
import seaborn as sns
sns.distplot(df["revenue"])

In [ ]:
def bins(x):
    for bar in range(20000, 820000, 20000):
        if x <= bar:
            return bar
# Créé une nouvelle colonne pour appliquer la fonction bins
df["rev_dist"] = df["revenue"].apply(lambda x: bins(x))

In [ ]:
df["count"] = 1
# Créé la table pivot de la distribution des revenues
pivot_table = pd.pivot_table(df, index = ["rev_dist"], values = ["count"], aggfunc = np.sum)

In [ ]:
# importer le modèle depuis SKLearn
from sklearn.cluster import KMeans
# K -clusters est égal à 3 parce que les produit seront classifié dans A, B ou C
kmeans = KMeans(n_clusters=3)
kmeans.fit(pivot_table)

In [ ]:
pivot_table["category"] = kmeans.labels_

In [ ]:
ABC_dict = {    
    0: "A",
    1: "C",
    2: "B"
}
pivot_table["ABC"] = pivot_table["category"].apply(lambda x: ABC_dict[x])

In [ ]:
df = pd.merge(df, pivot_table, on = "rev_dist", how ="left")

In [ ]:
df["count"] = 1
res = df.groupby('ABC').agg({'count': 'sum', 'revenue': 'sum'})
res['percent_articles'] = res['count'].transform(lambda x: (x/x.sum()) * 100)
res['percent_turnover'] = res['revenue'].transform(lambda x: (x/x.sum()) * 100)
print(res)